In [1]:
# File input/output: scipy.io
import numpy as np

In [2]:
# Loading and saving matlab files
from scipy import io as spio
a = np.ones((3, 3))
spio.savemat('file.mat', {'a': a}) # savemat expects a dictionary
data = spio.loadmat('file.mat', struct_as_record=True)
data['a']

NameError: name 'np' is not defined

In [3]:
# Reading images
from scipy import misc
misc.imread('fname.png')
import matplotlib.pyplot as plt
plt.imread('fname.png')

IOError: [Errno 2] No such file or directory: 'fname.png'

In [4]:
# Linear algebra operations: scipy.linalg


In [5]:
#The scipy.linalg.det() function computes the determinant of a square matrix

In [8]:
from scipy import linalg
import numpy as np
arr = np.array([[1, 2],[3, 4]])
linalg.det(arr)

-2.0

In [9]:
linalg.det(np.ones((3, 4)))

ValueError: expected square matrix

In [10]:
# The scipy.linalg.inv() function computes the inverse of a square matrix
arr = np.array([[1, 2],[3, 4]])
iarr = linalg.inv(arr)
np.allclose(np.dot(arr, iarr), np.eye(2))

True

In [11]:
# More advanced operations are available, for example singular-value decomposition (SVD)
arr = np.arange(9).reshape((3, 3)) + np.diag([1, 0, 1])
uarr, spec, vharr = linalg.svd(arr)
spec

array([ 14.88982544,   0.45294236,   0.29654967])

In [12]:
# The original matrix can be re-composed by matrix multiplication of the outputs of svd with np.dot
sarr = np.diag(spec)
svd_mat = uarr.dot(sarr).dot(vharr)
np.allclose(svd_mat, arr)

True

In [13]:
# Fast Fourier transforms: scipy.fftpack
# The scipy.fftpack module allows to compute fast Fourier transforms

In [14]:
time_step = 0.02
period = 5.
time_vec = np.arange(0, 20, time_step)
sig = np.sin(2 * np.pi / period * time_vec) + \
0.5 * np.random.randn(time_vec.size)

In [15]:
from scipy import fftpack
sample_freq = fftpack.fftfreq(sig.size, d=time_step)
sig_fft = fftpack.fft(sig)

In [16]:
pidxs = np.where(sample_freq > 0)
freqs = sample_freq[pidxs]
power = np.abs(sig_fft)[pidxs]

In [17]:
freq = freqs[power.argmax()]
np.allclose(freq, 1./period)  # check that correct freq is found

True

In [18]:
sig_fft[np.abs(sample_freq) > freq] = 0
main_sig = fftpack.ifft(sig_fft)

In [19]:
import pylab as plt
plt.figure()
plt.plot(time_vec, sig)
plt.plot(time_vec, main_sig, linewidth=3)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

C:\Anaconda\lib\site-packages\numpy\core\numeric.py:462: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [21]:
# Optimization and fit: scipy.optimize
# The scipy.optimize module provides useful algorithms for function minimization (scalar or multi-dimensional), curve fitting and root finding

In [22]:
from scipy import optimize

In [23]:
def f(x):
        return x**2 + 10*np.sin(x)

In [24]:
x = np.arange(-10, 10, 0.1)
plt.plot(x, f(x))
plt.show()

In [25]:
optimize.fmin_bfgs(f, 0)

Optimization terminated successfully.
         Current function value: -7.945823
         Iterations: 5
         Function evaluations: 24
         Gradient evaluations: 8


array([-1.30644003])

In [26]:
optimize.fmin_bfgs(f, 3, disp=0)

array([ 3.83746663])

In [27]:
optimize.basinhopping(f, 0)

                  nfev: 1746
 minimization_failures: 0
                   fun: -7.9458233756152845
                     x: array([-1.30644002])
               message: ['requested number of basinhopping iterations completed successfully']
                  njev: 582
                   nit: 100

In [28]:
xmin_local = optimize.fminbound(f, 0, 10)
xmin_local

3.8374671194983834

In [29]:
root = optimize.fsolve(f, 1)  # our initial guess is 1
root

array([ 0.])

In [30]:
root2 = optimize.fsolve(f, -2.5)
root2

array([-2.47948183])

In [31]:
# Curve fitting
xdata = np.linspace(-10, 10, num=20)
ydata = f(xdata) + np.random.randn(xdata.size)

In [32]:
def f2(x, a, b):
...     return a*x**2 + b*np.sin(x)

In [33]:
guess = [2, 2]
params, params_covariance = optimize.curve_fit(f2, xdata, ydata, guess)
params

array([  1.00453348,  10.0533622 ])

In [34]:
# Statistics and random numbers: scipy.stats

In [35]:
# Histogram and probability density function

In [36]:
a = np.random.normal(size=1000)
bins = np.arange(-4, 5)
histogram = np.histogram(a, bins=bins, normed=True)[0]
bins = 0.5*(bins[1:] + bins[:-1])
from scipy import stats
b = stats.norm.pdf(bins)  # norm is a distribution
plt.plot(bins, histogram)
plt.plot(bins, b)
plt.show()

In [37]:
# Interpolation: scipy.interpolate


In [38]:
measured_time = np.linspace(0, 1, 10)
noise = (np.random.random(10)*2 - 1) * 1e-1
measures = np.sin(2 * np.pi * measured_time) + noise

In [39]:
from scipy.interpolate import interp1d
linear_interp = interp1d(measured_time, measures)

In [40]:
computed_time = np.linspace(0, 1, 50)
linear_results = linear_interp(computed_time)
cubic_interp = interp1d(measured_time, measures, kind='cubic')
cubic_results = cubic_interp(computed_time)

In [41]:
# Numerical integration: scipy.integrate


In [42]:
from scipy.integrate import quad
res, err = quad(np.sin, 0, np.pi/2)
np.allclose(res, 1)
np.allclose(err, 1 - res)

True

In [43]:
def calc_derivative(ypos, time, counter_arr):
         counter_arr += 1
         return -2 * ypos

In [44]:
counter = np.zeros((1,), dtype=np.uint16)

In [45]:
from scipy.integrate import odeint
time_vec = np.linspace(0, 4, 40)
yvec, info = odeint(calc_derivative, 1, time_vec,args=(counter,), full_output=True)

In [46]:
counter

array([129], dtype=uint16)

In [48]:
info['nfe'][:10]

array([31, 35, 43, 49, 53, 57, 59, 63, 65, 69])

In [49]:
mass = 0.5  # kg
kspring = 4  # N/m
cviscous = 0.4  # N s/m
eps = cviscous / (2 * mass * np.sqrt(kspring/mass))
eps < 1
                  

True

In [50]:
nu_coef = cviscous / mass
om_coef = kspring / mass

In [51]:
def calc_deri(yvec, time, nuc, omc):
         return (yvec[1], -nuc * yvec[1] - omc * yvec[0])
time_vec = np.linspace(0, 10, 100)
yarr = odeint(calc_deri, (1, 0), time_vec, args=(nu_coef, om_coef))               

In [52]:
# Signal processing: scipy.signal

In [53]:
from scipy import signal
t = np.linspace(0, 5, 100)
x = t + np.random.normal(size=100)
plt.plot(t, x, linewidth=3)
plt.plot(t, signal.detrend(x), linewidth=3)

In [54]:
t = np.linspace(0, 5, 100)
x = np.sin(t)
plt.plot(t, x, linewidth=3)
plt.plot(t[::2], signal.resample(x, 50), 'ko')

In [56]:
# Image processing: scipy.ndimage
from scipy import ndimage

In [57]:
# Geometrical transformations on images
from scipy import misc
face = misc.face(gray=True)
shifted_face = ndimage.shift(face, (50, 50))
shifted_face2 = ndimage.shift(face, (50, 50), mode='nearest')
rotated_face = ndimage.rotate(face, 30)
cropped_face = face[50:-50, 50:-50]
zoomed_face = ndimage.zoom(face, 2)
zoomed_face.shape

(1536L, 2048L)